In [1]:
%matplotlib inline
%env KERAS_BACKEND=tensorflow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from ipywidgets import interact, IntSlider, Button
from ipywidgets import interact_manual
# import sympy as sym
# from sympy import S,N
# import locale # 調整數字格式

# from sklearn.linear_model import LinearRegression
from keras.utils import np_utils #one hot encoding
from keras.models import Sequential #神經網路的框架keras.models
from keras.layers import Dense, Activation, Embedding
from keras.layers import LSTM
# from keras.layers import Flatten # 將資料展平
# from keras.layers import Conv2D, MaxPool2D, MaxPooling2D
from keras.optimizers import SGD, Adadelta, Adam
from keras.models import model_from_json
# from keras.datasets import mnist # mnist內 包含手寫測試資料
from keras.datasets import imdb # imdb內 包含電影評論資料
from keras.preprocessing import sequence # 用來同整不同長度的電影評論，多的擷掉，少的補 0
# from keras.preprocessing.text import Tokenizer # 轉換文字成數字
# from sklearn.model_selection import train_test_split

# #for Functional API
# from keras.models import Model
# from keras.layers import Input
# from keras.layers import concatenate, add
# #for Functional API transfer learning
# from keras.layers.core import Lambda
# from keras import backend as K

# from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout

# import matplotlib.image as mpimg #import image file

env: KERAS_BACKEND=tensorflow


Using TensorFlow backend.


In [2]:
INDEX_FROM = 3 # 預設本來就是 3 ，意思是 word to int 的 int一定大於等於 3
NUM_WORDS = 10000 # num_words 代表要擷取多少常用字，其餘的字當作不認識
(x_train0, y_train0), (x_test0, y_test0) = imdb.load_data(num_words = NUM_WORDS, index_from=INDEX_FROM)
word_to_id = imdb.get_word_index()
word_to_id = {k:(v+INDEX_FROM) for k,v in word_to_id.items()}
word_to_id["<PAD>"] = 0
word_to_id["<START>"] = 1
word_to_id["<UNK>"] = 2
id_to_word = {value:key for key,value in word_to_id.items()}
id_to_word[0] = ""
id_to_word[1] = ""
id_to_word[2] = "***"

def convert_inverse_to_text(list_of_int):
    return ' '.join(id_to_word[id] for id in list_of_int )

print("There are %d training data" %x_train0.shape)
print("There are %d testing  data" %x_test0.shape)
print("There are %d training label" %y_train0.shape)
print("There are %d testing  label" %y_test0.shape)
print("\nthe text of x_train0[0] = \n", convert_inverse_to_text(x_train0[0]), " ...\n\ny_train0[0] = \n", y_train0[0])

x_train = sequence.pad_sequences(x_train0, maxlen = 150)
x_test = sequence.pad_sequences(x_test0, maxlen = 150)
y_train = y_train0
y_test = y_test0
# y_train = np_utils.to_categorical(y_train0, 2)
# y_test = np_utils.to_categorical(y_test0, 2)
# x_train.shape

There are 25000 training data
There are 25000 testing  data
There are 25000 training label
There are 25000 testing  label

the text of x_train0[0] = 
  this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert *** is an amazing actor and now the same being director *** father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for *** and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also *** to the two little boy's that played the *** of norman and paul they were just brilliant children are often left out of the *** list i think because the star

In [3]:
N = 500 # 希望用 Embedding 來做 dimension reduction 將資料降至N維
K = 30 # 決定LSTM 神經元個數 K
# 有人說：當 N >> K 時會提升相關的準確率？？？

model=Sequential()
model.add(Embedding(NUM_WORDS, N))
model.add(Dropout(0.35))
model.add(LSTM(K))
model.add(Dropout(0.25))
# model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 500)         5000000   
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 500)         0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 30)                63720     
_________________________________________________________________
dropout_2 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 31        
Total params: 5,063,751
Trainable params: 5,063,751
Non-trainable params: 0
___________

### LSTM cell
黃色的地方(共4個)自己就有 weight 跟 bias  
輸入有 $h_{t-1}$ 跟 $x_t$ ，共(K+N)個輸入  
也就是~1個黃色的地方就會有 K+N 個輸入的weight + 1個bias  
所以一個 LSTM cell 就會有 4*(K+N+1)個params

4240 = 4(K+N+1)K

<!-- <div style="width: 500px; height: 240px; overflow: hidden"> -->
<img src="https://cdn-images-1.medium.com/max/900/1*ZX2mVCwMIOhftEaf4FTOYQ.png" width="500" title="LSTM">
<!-- <\div> -->

### 更多關於 RNN vs GRU vs LSTM
[Simple RNN vs GRU vs LSTM :- Difference lies in More Flexible control](https://medium.com/@saurabh.rathor092/simple-rnn-vs-gru-vs-lstm-difference-lies-in-more-flexible-control-5f33e07b1e57)

In [4]:
model.fit(x_train, y_train, batch_size=32, epochs=2)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
25000/25000 [==============================] - 139s 6ms/step - loss: 0.4005 - acc: 0.8203
Epoch 2/2
25000/25000 [==============================] - 146s 6ms/step - loss: 0.2400 - acc: 0.9066


In [5]:
score = model.evaluate(x_test, y_test)
print('loss:', score[0])
print('正確率', score[1])

25000/25000 [==============================] - 26s 1ms/step
loss: 0.3210943642377853
正確率 0.86788
